# Neeps- Medium

In [1]:
import getpass
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
pwd = getpass.getpass()
engine = create_engine(
    'postgresql+psycopg2://postgres:%s@192.168.31.31:15432/sqlzoo' % (pwd))
pd.set_option('display.max_rows', 60)


········


In [2]:
ut_staff = pd.read_sql_table('ut_staff', engine)
ut_student = pd.read_sql_table('ut_student', engine)
ut_event = pd.read_sql_table('ut_event', engine)
ut_room = pd.read_sql_table('ut_room', engine)
ut_attends = pd.read_sql_table('ut_attends', engine)
ut_teaches = pd.read_sql_table('ut_teaches', engine)
ut_occurs = pd.read_sql_table('ut_occurs', engine)
ut_modle = pd.read_sql_table('ut_modle', engine)
ut_week = pd.read_sql_table('ut_week', engine)


## 6.
**Show the 'size' of each of the co72010 events. Size is the total number of students attending each event.**

In [3]:
a = (ut_event.rename(columns={'id': 'event_id'})
     .loc[ut_event['modle']=='co72010']
     .merge(ut_attends, left_on='event_id', right_on='event')
     .merge(ut_student, left_on='student', right_on='id'))
(a.groupby('event_id')['sze'].sum().reset_index()
 .sort_values('sze', ascending=False))

,event_id,sze
0,co72013.L01,215
3,co72013.T02,49
5,co72013.T04,45
2,co72013.T01,40
6,co72013.T05,35
7,co72013.T06,29
1,co72013.L02,10
4,co72013.T03,10


## 7.
**For each post-graduate module, show the size of the teaching team. (post graduate modules start with the code co7).**

In [4]:
t = (ut_teaches.merge(ut_event.rename(columns={'id': 'evt_id'}), 
                      left_on='event', right_on='evt_id')
     .merge(ut_modle.loc[ut_modle['id'].str.contains('^co7')], 
            left_on='modle', right_on='id')
    [['id', 'staff']].drop_duplicates())
t.groupby('id')['staff'].count().reset_index()


,id,staff
0,co72002,1
1,co72003,2
2,co72004,1
3,co72005,2
4,co72006,1
5,co72010,2
6,co72011,2
7,co72012,1
8,co72016,2
9,co72017,1


## 8.
**Give the full name of those modules which include events taught for fewer than 10 weeks.**

In [5]:
a = (ut_modle.rename(columns={'name': 'modle_name'})
     .merge(ut_event.rename(columns={'id': 'evt_id'}), 
            left_on='id', right_on='modle')
     .merge(ut_occurs, left_on='evt_id', right_on='event'))
b = (a.groupby(['evt_id', 'modle_name'])['event'].count()
     .reset_index().sort_values('modle_name'))
b.loc[b['event']<10, ['modle_name']].drop_duplicates()


,modle_name
127,Interactivity and the Internet
149,Internet Multimedia
102,Languages and Algorithms
109,Project


## 9.
**Identify those events which start at the same time as one of the co72010 lectures.**

In [6]:
t = ut_event.merge(ut_occurs, left_on='id', right_on='event')
t['time'] = t[['week', 'dow', 'tod']].agg('-'.join, axis=1)
(t.loc[t['modle'] != 'co72010'].merge(t.loc[t['modle'] == 'co72010'], on='time')[['id_x']]
 .sort_values('id_x').drop_duplicates())

,id_x
0,co12004.T04
62,co12004.T05
162,co12005.T01
192,co12005.T04
45,co12006.L03
133,co12008.L01
6,co12012.T01
31,co22005.T02
280,co22005.T04
217,co22005.T07


## 10.
**How many members of staff have contact time which is greater than the average?**

In [7]:
t = (ut_event.merge(ut_teaches, left_on='id', right_on='event')
     .merge(ut_occurs, left_on='id', right_on='event')
     .groupby('staff')['duration'].sum().reset_index())
pd.DataFrame({'count': [
    t.loc[t['duration'] > sum(t['duration'])/ut_teaches['staff'].nunique()].shape[0]]})


,count
0,19
